In [13]:
import sqlite3
import re
from pymongo import MongoClient

def region_clean(reg):
    # Strip place from after comma
    reg = reg.split(', ')[-1]
    
    # Remove dashes and extra space created by this
    reg = reg.replace('-', ' ')
    reg = reg.replace('  ', ' ')
    
    # Fixing an issue with when Philippine Islands gets stripped
    reg = reg.replace('Philippine ', 'Philippines ')
    
    # Remove off offshore of near include the coast but not including 
    # certain variations
    reg = re.sub(r'\b(?<!\w)(?:offshore|near|off|of(?!\sthe\sCongo|\sMicronesia|\sCalifornia|\sJapan|\sOkhotsk))?(?:the(?!\sCongo)|the\scoast|coast\sof)?\b(?!\w)','', reg)
    
    # Remove cardinal directions
    reg = re.sub(r'\b(?<!\w)(?:[nN]orth|[sS]outh|[eE]ast|[wW]est)?(?:east|west|ern)?(?:ern)?\b(?!\w)','', reg)

    # Remove additional location descriptions
    reg = re.sub(r'\b(?<!\w)(?:[cC]entral|[nN]ear the|[sS]trait)?(?:coast of)?\b(?!\w)','', reg)
    
    # Remove islands and regions
    reg = re.sub(r'\b(?<!\w)(?:[iI]sland|[rR]egion)s?\b(?!\w)','', reg)
    
    # Turn abbreviated names into correct full names
    regions = {
        'CA': 'California',
        'MX': 'Mexico',
    }
    
    for (key, value) in regions.items():
        if reg == key:
            reg = value

    return reg.strip()

conn = sqlite3.connect('earthquakes.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS earthquakes;
CREATE TABLE earthquakes (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    magnitude FLOAT,
    time      INTEGER,
    depth     FLOAT,
    place     TEXT,
    title     TEXT, 
    felt      INTEGER,
    region    TEXT 
);
''')

# Connect to MongoDB cluster
uri = "mongodb+srv://<user>:<password>@cluster0-hsnh5.mongodb.net/test?retryWrites=true&w=majority"
client = MongoClient(uri)

# Connect to earthquakesdb and loop through data
with client:
    
    db = client['earthquakesdb']
    eq = db.earthquakes.find()
    
    for e in eq:
        magnitude = e['properties']['mag']
        time      = e['properties']['time']
        depth     = e['geometry']['coordinates'][2]
        place     = e['properties']['place']
        title     = e['properties']['title']
        felt      = e['properties']['felt']
        
        region    = region_clean(place)
            
        cur.execute('''INSERT OR REPLACE INTO earthquakes
            (magnitude, time, depth, place, title, felt, region) 
            VALUES ( ?, ?, ?, ?, ?, ?, ? )''', 
            ( magnitude, time, depth, place, title, felt, region ) )
    

    # Periodically print out data and commit so you know you are making progress
    # and you do not lose data if the program crashes
    cnt = 0
    
    if cnt%50 == 0 : 
        print(cnt, title)    
        conn.commit()
        
    cnt += 1

# VIEW SOME DATA INSERTED
sqlstr = '''SELECT id, magnitude, time, depth, place, title, felt, region 
           FROM earthquakes WHERE id <= 5
           ORDER BY title '''

print
for row in cur.execute(sqlstr) :
   print(row)

cur.close()

0 M 2.9 - Central Alaska
(4, 2.5, 1074638275390, -0.107, 'Washington', 'M 2.5 Explosion - Washington', None, 'Washington')
(5, 2.8, 1074619263280, 4.428, '7km ESE of Santa Clarita, CA', 'M 2.8 - 7km ESE of Santa Clarita, CA', None, 'California')
(2, 3.3, 1074616717400, 83.0, 'Coquimbo, Chile', 'M 3.3 - Coquimbo, Chile', None, 'Chile')
(1, 4.5, 1074641238570, 36.2, 'Fox Islands, Aleutian Islands, Alaska', 'M 4.5 - Fox Islands, Aleutian Islands, Alaska', None, 'Alaska')
(3, 4.6, 1074617195460, 10.0, 'southeast of the Loyalty Islands', 'M 4.6 - southeast of the Loyalty Islands', None, 'Loyalty')
